<a href="https://www.kaggle.com/code/kitsewio/smoke-fire-detection-w-yolov8?scriptVersionId=229628301" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 📌If the laptop seemed useful to you, please do not forget to vote!❤️

  🎯 This notebook presents work with YOLOv8. The result of the training was obtained according to mAP50: 0.658 - for all classes, 0.717 - for smoke, 0.599 - for fire ***in just 10 epochs***!

# 📥 Loading required libraries

In [ ]:
!pip install ultralytics

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
import yaml
import torch
from PIL import Image
from IPython.display import display, Video
from ultralytics import YOLO
from collections import deque
import subprocess

# Paths to datasets

In [ ]:
test_dir = "/kaggle/input/smoke-fire-detection-yolo/data/test"
train_dir = "/kaggle/input/smoke-fire-detection-yolo/data/train"
val_dir = "/kaggle/input/smoke-fire-detection-yolo/data/val"
annot_dir = "/kaggle/input/smoke-fire-detection-yolo/data.yaml"

# 📝 Creating a file .yaml

In the original data file.yaml is the wrong way to the dataset, so it was decided to create a new one without unnecessary characters.

In [ ]:
classes = ['smoke', 'fire']

data_yaml_content = f"""
train: {train_dir}
val: {val_dir}
test: {test_dir}

nc: {len(classes)}
names: {classes}
"""

yaml_path = "/kaggle/working/data.yaml"
with open(yaml_path, "w") as f:
    f.write(data_yaml_content)

!cat {yaml_path}

# 📥 Loading the model

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
device=torch.device(0)
model.to(device)
device

# 🦾 Training a pre-trained model

In [ ]:
history = model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    pretrained=True,
    batch=32,
    device=device
)

Here we can see what learning outcomes the model has achieved.

# 📈 Visualization of training

Loss naturally dropped throughout the entire training, and accuracy increased.

In [ ]:
train_results = "runs/detect/train2/results.png"

if os.path.exists(train_results):
    img = Image.open(train_results)
    display(img)
else:
    print("Training results image not found.")

# 👀 Visualization of the model's operation

In [ ]:
import glob

In [ ]:
all_images = glob.glob("/kaggle/input/smoke-fire-detection-yolo/data/test/images/*.jpg")

test_images = random.sample(all_images, 6)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for img_path, ax in zip(test_images, axes.flatten()):
    results = model(img_path)      
    result_img = results[0].plot() 
    
    img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    ax.axis("off")
    ax.set_title(os.path.basename(img_path))  

plt.tight_layout()
plt.show()

In [ ]:
metrics = model.val(data=yaml_path, split="test")